In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings('ignore')

In [3]:
df= pd.read_csv("AirPassengers.csv")
df.head()

,Month,#Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [4]:
passengers = df['#Passengers'].values   #Sadece passengers değerleri alınır.

In [7]:
#RNN veriyi küçük aralıklarla daha iyi öğrenir. O yüzden Normalize işlemi yapılır.
scaler = MinMaxScaler(feature_range=(0,1))
passengers_scaled = scaler.fit_transform(passengers.reshape(-1,1))

In [9]:
#Yolcu kategorilerini 3 farklı kategorilere ayrılır. (Çok sınıflı bir problem olmasını istediğimiz için)
thresholds = np.percentile(passengers,[33,66]) #0-33,33-66,66-100 şeklinde ayrılır.
def categorize(value):
    if value <= thresholds[0]:
        return 0   # düşük talep kategorisi
    elif value <= thresholds[1]:
        return 1 #orta talep
    else:
        return 2 #yüksek talep

labels = np.array([categorize(v) for v in passengers]) #Çıktısı 0,1,2 olacak değerleri arraya çevrilir. RNN ve kerası uygulayabilmek için.

In [12]:
#Geçmiş 12 ayın verisi ile gelecek ayın kategorisi tahmin edilir.
time_steps = 12

#Girdi= X, Çıkış(label)= y
X,y= [],[]
for i in range(len(passengers_scaled)- time_steps): 
    X.append(passengers_scaled[i:i+time_steps]) #i değerinden başlayarak 12 aylık veri alınır.i= 0 ise ay 0-11
    y.append(labels[i+time_steps]) #12. aydan sonraki ayın kategorisini tutar. i+12

X =np.array(X)  #Listeler tekrar diziye çevrilir.
y=np.array(y)

#Softmax kullanılacağı için etiketler one-hot formatına çevrilir. 
#Kategori 0 → [1,0,0], 1 → [0,1,0], 2 → [0,0,1]
y= to_categorical(y,num_classes = 3)

#Rnn girişi 3 boyutlu olmalıdır. 
#X.shape[0] → örnek sayısı
#X.shape[1] → zaman adımı (12 ay)
#1 → her adımda 1 özellik (yolcu sayısı)

X=X.reshape((X.shape[0],X.shape[1],1))

In [16]:
#Model kurma

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense,Flatten,Dropout
import warnings
warnings.filterwarnings('ignore')

model= Sequential()

In [17]:
#Giriş katmanı
model.add(SimpleRNN(64, return_sequences=True, input_shape=(time_steps, 1)))

model.add(Flatten())
model.add(Dense(32,activation = 'relu'))
model.add(Dropout(0.2))     #Eğitim olurken nöronların %20'si rastgelele kapanır.

#Çıkış katmanı
model.add(Dense(3,activation='softmax'))

In [18]:
#model derleme
model.compile(optimizer ='adam',loss = 'categorical_crossentropy',metrics=['accuracy'])

In [44]:
#Model eğitme
history = model.fit(X,y,epochs= 50,batch_size = 16,verbose =1) #verbose:Eğitim sırasındaki çıktı durumunu gösterir.


Epoch 1/50
9/9 [==============================] - 0s 3ms/step - loss: 0.0947 - accuracy: 0.9545
Epoch 2/50
9/9 [==============================] - 0s 4ms/step - loss: 0.0800 - accuracy: 0.9621
Epoch 3/50
9/9 [==============================] - 0s 4ms/step - loss: 0.1350 - accuracy: 0.9545
Epoch 4/50
9/9 [==============================] - 0s 4ms/step - loss: 0.1725 - accuracy: 0.9318
Epoch 5/50
9/9 [==============================] - 0s 4ms/step - loss: 0.0777 - accuracy: 0.9773
Epoch 6/50
9/9 [==============================] - 0s 4ms/step - loss: 0.1013 - accuracy: 0.9545
Epoch 7/50
9/9 [==============================] - 0s 4ms/step - loss: 0.0803 - accuracy: 0.9621
Epoch 8/50
9/9 [==============================] - 0s 3ms/step - loss: 0.0815 - accuracy: 0.9545
Epoch 9/50
9/9 [==============================] - 0s 3ms/step - loss: 0.0678 - accuracy: 0.9773
Epoch 10/50
9/9 [==============================] - 0s 3ms/step - loss: 0.0890 - accuracy: 0.9621
Epoch 11/50
9/9 [======================

In [45]:
preds = model.predict(X[:100])  # İlk 5 örnek için tahmin yap
print("Olasılık Dağılımları:\n", preds)
print("Tahmin Sınıfları:", np.argmax(preds, axis=1))  # En yüksek olasılıklı sınıf


4/4 [==============================] - 0s 2ms/step
Olasılık Dağılımları:
 [[9.99999940e-01 4.02444106e-10 1.10291403e-11]
 [9.99999940e-01 7.65010999e-10 1.76075456e-11]
 [9.99999940e-01 1.97448777e-08 3.35042105e-10]
 [9.99999940e-01 1.90290699e-08 2.19140872e-10]
 [9.99999940e-01 7.52853690e-09 3.56526510e-11]
 [9.99999702e-01 2.48016505e-07 5.19774168e-10]
 [9.99996722e-01 3.26396844e-06 8.45071035e-09]
 [9.99997318e-01 2.59985268e-06 1.22440902e-08]
 [9.99999821e-01 1.12627056e-07 2.51912868e-09]
 [9.99999940e-01 2.11893725e-09 9.13039505e-11]
 [9.99999940e-01 2.28908087e-10 4.67887621e-12]
 [9.99999940e-01 2.77874257e-09 5.65278969e-11]
 [9.99999940e-01 4.97821173e-10 1.36301890e-11]
 [9.99999940e-01 2.14547735e-09 4.80249590e-11]
 [9.99999821e-01 1.16230929e-07 1.49886470e-09]
 [9.99999821e-01 6.39786393e-08 4.71892414e-10]
 [9.99999940e-01 2.71856084e-08 3.69657777e-11]
 [9.99971807e-01 2.81073790e-05 4.40452874e-09]
 [9.91896927e-01 8.10212269e-03 9.09518803e-07]
 [9.95919406e-